# 멀티모달 데이터 처리 - Amazon Bedrock Knowledge Bases를 활용한 텍스트·이미지 엔드 투 엔드 예제

멀티모달 RAG는 이미지, 차트, 다이어그램, 표 등 시각 데이터와 텍스트 데이터를 함께 분석해 인사이트를 도출할 수 있습니다. Bedrock Knowledge Bases는 자체 데이터 소스의 컨텍스트 정보를 활용해 정확도 높고 지연 시간이 낮으며 안전하고 사용자 맞춤형인 생성형 AI 애플리케이션을 구축할 수 있는 엔드 투 엔드 관리형 RAG 워크플로를 제공합니다.

Bedrock Knowledge Bases는 텍스트와 시각 데이터를 모두 추출하고, 선택한 임베딩 모델로 의미 임베딩을 생성해 지정한 벡터 스토어에 저장합니다. 이를 통해 텍스트뿐 아니라 시각 데이터에서 파생된 질문에도 응답을 검색하고 생성할 수 있습니다. 또한 검색 결과에는 시각 데이터에 대한 출처 정보가 포함되어 생성된 출력물의 투명성과 신뢰성을 높입니다.

현재 프리뷰 중인 관리형 서비스 Amazon Bedrock Data Automation을 사용해 멀티모달 데이터를 자동으로 추출하거나, 기본 프롬프트를 커스터마이징할 수 있는 Claude 3.5 Sonnet, Claude 3 Haiku와 같은 Foundation Model을 선택할 수 있습니다.

이 노트북은 Amazon Bedrock Knowledge Bases를 사용해 멀티모달 RAG를 구축하는 샘플 코드를 제공합니다.

#### 단계: 
- S3에서 데이터를 읽고 쓰고 필요한 Foundation Model에 접근하기 위한 권한을 포함한 Knowledge Base 실행 역할을 생성합니다.
- 다양한 콘텐츠가 포함된 문서로 Knowledge Base를 생성합니다.
- Knowledge Base 안에 데이터 소스를 생성합니다.
- KB API를 사용해 ingestion 작업을 시작합니다. 이 작업은 데이터 소스에서 문서를 읽고, Bedrock Data Automation 또는 Foundation Model을 사용해 문서(이미지, 차트, 표 등)를 파싱하고, 청크로 분할한 뒤 Amazon Titan Embeddings 모델로 임베딩을 생성해 AOSS에 저장합니다. 이 모든 과정을 데이터 파이프라인을 구축·배포·운영하지 않고 수행할 수 있습니다.

데이터가 Bedrock Knowledge Base에 준비되면 Amazon Bedrock이 제공하는 Knowledge Base API를 사용해 질문 응답 애플리케이션을 구축할 수 있습니다.

#### 사전 준비 사항:

Amazon Bedrock 콘솔에서 `Anthropic Claude 3 Sonnet`, `Amazon Nova Micro`, `Titan Text Embeddings V2` 모델 접근 권한을 활성화하세요.

<div class="alert alert-block alert-info">
<b>참고:</b> "Run All Cells" 옵션 대신 노트북 셀을 하나씩 실행하세요.
</div>

### 0 - 설정
나머지 셀을 실행하기 전에 아래 셀을 먼저 실행해 필요한 라이브러리를 설치하고 Bedrock에 연결하세요.

라이브러리를 설치하는 동안 pip 의존성 오류가 표시되더라도 무시해도 됩니다.

In [ ]:
# %pip install --upgrade pip --quiet
# %pip install -r ../../requirements.txt --no-deps --quiet
# %pip install -r ../../requirements.txt --upgrade --quiet

In [ ]:
# %pip install --upgrade boto3
import boto3
print(boto3.__version__)

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
# Print sys.path to verify
# print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

In [ ]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

In [ ]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"

knowledge_base_name = f"bedrock-multi-modal-kb-{suffix}"
knowledge_base_description = "Multi-modal RAG knowledge base."

bucket_name = f'{knowledge_base_name}-{account_id}'
# intermediate_bucket_name = f'{knowledge_base_name}-mm-storage-{account_id}'
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

#### 멀티모달 Knowledge Base에는 S3, SharePoint 등 여러 데이터 소스를 추가할 수 있습니다. 이 노트북에서는 S3 버킷을 사용해 Knowledge Base 생성을 테스트합니다.

각 데이터 소스에는 서로 다른 사전 준비 사항이 있을 수 있으므로 자세한 내용은 AWS 문서를 참고하세요.

In [ ]:
## Please uncomment the data sources that you want to add and update the placeholder values accordingly.

data_sources=[
                {"type": "S3", "bucket_name": bucket_name}, 

                # {"type": "SHAREPOINT", "tenantId": "888d0b57-69f1-4fb8-957f-e1f0bedf64de", "domain": "yourdomain",
                #   "authType": "OAUTH2_CLIENT_CREDENTIALS",
                #  "credentialsSecretArn": f"arn:aws::secretsmanager:{region_name}:secret:<<your_secret_name>>",
                #  "siteUrls": ["https://yourdomain.sharepoint.com/sites/mysite"]
                # },
            ]
                
pp = pprint.PrettyPrinter(indent=2)

### 1 - 멀티모달 Knowledge Base 생성

In [ ]:
# For multi-modal RAG While instantiating BedrockKnowledgeBase, pass multi_modal= True and choose the parser you want to use

knowledge_base = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}',
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    multi_modal= True,
    parser='BEDROCK_FOUNDATION_MODEL', # BEDROCK_DATA_AUTOMATION
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{suffix}-f'
)

### 2 - 데이터 적재
공개적으로 제공되는 리치 콘텐츠 PDF를 다운로드해 S3 버킷에 업로드합니다

In [ ]:
import os

def create_directory(directory_name):    
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
    else:
        print(f"Directory '{directory_name}' already exists.")

# Call the function to create the directory
create_directory("mm-data")

In [ ]:
import requests

def download_file(url, filename):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Open the file in write-binary mode
        with open(filename, 'wb') as file:
            # Write the content of the response to the file
            file.write(response.content)
        print(f"File downloaded successfully: {filename}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

# URL of the file to download
url = "https://sgp.fas.org/crs/misc/IF12695.pdf"

# Name for the downloaded file
filename = "./mm-data/tornadoes_report.pdf"

# Call the function to download the file
download_file(url, filename)

##### 데이터를 S3 버킷 데이터 소스로 업로드

In [ ]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("./mm-data", bucket_name)

### Ingestion 작업 시작
KB와 데이터 소스를 생성한 뒤 각 데이터 소스에 대해 ingestion 작업을 시작할 수 있습니다.
ingestion 작업 동안 KB는 데이터 소스에서 문서를 가져와 텍스트를 추출하고, 지정한 청킹 크기에 따라 분할하며, 각 청크의 임베딩을 생성해 이번에는 OSS 벡터 데이터베이스에 저장합니다.

참고: 현재는 한 번에 하나의 ingestion 작업만 실행할 수 있습니다.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()

In [ ]:
# keep the kb_id for invocation later in the invoke request
kb_id = knowledge_base.get_knowledge_base_id()
%store kb_id

### 4 - Knowledge Base 테스트
Knowledge Base가 준비되었으므로 [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html)와 [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) 함수를 사용해 확인할 수 있습니다. 

#### Retrieve and Generate API로 Knowledge Base 테스트

먼저 retrieve and generate API로 Knowledge Base를 시험해 보겠습니다. 이 API는 Bedrock이 Knowledge Base에서 필요한 참조를 검색하고 Bedrock의 Foundation Model로 최종 답변을 생성합니다.

query = `Summarize annual trends of tornado reports and how it varies year over year.`

이 질의에 대한 올바른 응답은 PDF 문서의 차트/그래프에서 가져와야 합니다.

In [ ]:
query = "Summarize annual trends of tornado reports and how it varies year over year."

In [ ]:
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
# foundation_model = "amazon.nova-micro-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

In [ ]:
from PIL import Image
import s3fs

fs = s3fs.S3FileSystem()

## Function to print retrieved response

def print_response(response):
#structure 'retrievalResults': list of contents. Each list has ['ResponseMetadata', 'citations', 'output', 'sessionId']
    print( f'OUTPUT: {response["output"]["text"]} \n')
    
    print(f'CITATION DETAILS: \n')
    
    for num, chunk in enumerate(response['citations']):
        print(f'CHUNK {num}',end='\n'*1)
        print("========")
        print(f'\t Generated  Response Text: ')
        print(f'\t ------------------------- ')
        print(f'\t Generated  Response Text: ',chunk['generatedResponsePart']['textResponsePart']['text'],end='\n'*2)
        for i, ref in enumerate (chunk['retrievedReferences']):
            print(f'\t Retrieved References: ')
            print(f'\t ---------------------', )
            print(f'\n\t\t --> Location:', ref['location'])
            print(f'\t\n\t\t --> Metadata: \n\t\t\t ---> Source', ref['metadata']['x-amz-bedrock-kb-source-uri'])
            # print(f'\t\n\t\t\n\t\t\t ---> x-amz-bedrock-kb-description', ref['metadata']['x-amz-bedrock-kb-description'])
            print(f'\t\n\t\t\n\t\t\t ---> x-amz-bedrock-kb-byte-content-source', ref['metadata']['x-amz-bedrock-kb-byte-content-source'])
            print("")
            with fs.open(ref['metadata']['x-amz-bedrock-kb-byte-content-source']) as f:
                display(Image.open(f).resize((400, 400)))

In [ ]:
print_response(response)

#### Retrieve API로 Knowledge Base 테스트
추가적인 제어가 필요하다면 Retrieve API를 사용해 질의와 가장 잘 일치하는 청크를 직접 가져올 수 있습니다. 이 설정에서는 원하는 결과 수를 구성하고 자체 애플리케이션 로직으로 최종 답변을 제어할 수 있습니다. API는 일치하는 콘텐츠, 해당 S3 위치, 유사도 점수, 청크 메타데이터를 제공합니다.

In [ ]:
response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        "text": "How many new positions were opened across Amazon's fulfillment and delivery network?"
    }
)

def response_print(retrieve_resp):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        if 'text' in chunk['content']:
            print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        if 'byteContent' in chunk['content']:
            print(f'Chunk {num}: ',chunk['content']['byteContent'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)
        print("--------------------------------")

response_print(response_ret)

### 정리
아래 섹션의 주석을 해제한 뒤 실행해 모든 리소스를 삭제하세요.

In [ ]:
# delete local directory
import shutil

dir_path = "mm-data" # Replace with the actual path

try:
    shutil.rmtree(dir_path)
    print(f"Directory '{dir_path}' and its contents have been deleted successfully.")
except FileNotFoundError:
    print(f"Directory '{dir_path}' not found.")
except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
# # Delete resources
# print("===============================Deleteing resources ==============================\n")
knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)